In [1]:
import gb_mep

In [5]:
import pickle
import numpy as np
import pandas as pd
santander_training = pd.read_csv('../data/santander_train.csv') 
santander_test = pd.read_csv('../data/santander_test.csv') 
santander_distances = np.load('../data/santander_distances.npy')


In [6]:
santander_training

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,start_id,end_id,start_time,end_time
0,117501471,780,8657,02/03/2022 00:13,501.0,"Cephas Street, Bethnal Green",02/03/2022 00:00,278,"Tooley Street, Bermondsey",722,131,0.483492,13.192747
1,117501472,1380,7261,02/03/2022 00:23,376.0,"Millbank Tower, Pimlico",02/03/2022 00:00,596,"Parson's Green , Parson's Green",531,460,0.484768,23.477447
2,117501473,240,19838,02/03/2022 00:04,362.0,"Royal College Street, Camden Town",02/03/2022 00:00,797,"Ossulston Street, Somers Town",515,603,0.624303,4.196019
3,117501474,780,18705,02/03/2022 00:13,478.0,"Stepney Green Station, Stepney",02/03/2022 00:00,217,"Wormwood Street, Liverpool Street",804,689,0.807900,13.604670
4,117501478,1800,10602,02/03/2022 00:31,700.0,"Battersea Church Road, Battersea",02/03/2022 00:01,74,"Vauxhall Cross, Vauxhall",740,48,1.568787,31.912915
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6453258,124122193,420,21790,16/08/2022 23:59,665.0,"Smugglers Way, Wandsworth",16/08/2022 23:52,735,"Grant Road East, Clapham Junction",302,637,241912.321072,241919.498489
6453259,124122190,420,12350,16/08/2022 23:59,676.0,"Hartington Road, Stockwell",16/08/2022 23:52,74,"Vauxhall Cross, Vauxhall",740,342,241912.686823,241919.489431
6453260,124122202,240,19139,16/08/2022 23:57,770.0,"Gwendwr Road, West Kensington",16/08/2022 23:53,595,"Hammersmith Road, Hammersmith",327,324,241913.941339,241917.596443
6453261,124122207,360,18066,16/08/2022 23:59,368.0,"Harriet Street, Knightsbridge",16/08/2022 23:53,424,"Ebury Bridge, Pimlico",224,337,241913.962367,241919.498538
